In [5]:
%pip install urllib
%pip install pyodbc
%pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
    --------------------------------------- 0.0/2.1 MB 1.4 MB/s eta 0:00:02
   ------ --------------------------------- 0.3/2.1 MB 4.1 MB/s eta 0:00:01
   ---------------- ----------------------- 0.9/2.1 MB 7.8 MB/s eta 0:00:01
   --------------------------- ------------ 1.4/2.1 MB 9.2 MB/s eta 0:00:01
   -------------------------------------- - 2.0/2.1 MB 9.8 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 9.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/297.0 kB ? eta -:--:--
   --------------------------------------- 297.0/297.0 kB 17.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


**Step 1: Define your connection details**

In [24]:
server='DESKTOP-HJVSCEN\MSSQLSERVER1'
database='Python ETL'
username='sa'
password='Ka@12345678'

**Step 2: Build the full connection string (ODBC-style)**

In [25]:
import urllib

ConnectionString = f"""
    DRIVER={{ODBC Driver 18 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""
# URL-encode the connection string for SQLAlchemy
params=urllib.parse.quote_plus(ConnectionString)
print(params)

%0A++++DRIVER%3D%7BODBC+Driver+18+for+SQL+Server%7D%3B%0A++++SERVER%3DDESKTOP-HJVSCEN%5CMSSQLSERVER1%3B%0A++++DATABASE%3DPython+ETL%3B%0A++++UID%3Dsa%3B%0A++++PWD%3DKa%4012345678%3B%0A++++TrustServerCertificate%3Dyes%3B%0A


**Step 3: Create the SQLAlchemy engine**

In [26]:
from sqlalchemy import create_engine

engine=create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

**Step 4: Insert your CSV using pandas.to_sql()**

In [36]:
import pandas as pd

df=pd.read_csv('../Datasets/transaction_data.csv')
df.isna().sum()
df.dtypes

transaction_id        int64
customer_id           int64
amount              float64
transaction_date     object
product_category     object
payment_method       object
store_location       object
dtype: object

In [38]:

df['amount'].fillna(df['amount'].mean(),inplace=True)
df

C:\Users\Tanuj\AppData\Local\Temp\ipykernel_15868\3527743662.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['amount'].fillna(df['amount'].mean(),inplace=True)


,transaction_id,customer_id,amount,transaction_date,product_category,payment_method,store_location
0,1,565,2992.470000,2025-03-10 01:20:54,Sports,Debit Card,New York
1,2,323,2041.870000,2025-01-02 15:24:19,Clothing,Cash,New York
2,3,398,107.350000,2025-02-16 03:49:01,Beauty,Debit Card,Online
3,4,19,2532.550126,2025-04-30 15:26:23,Sports,Debit Card,Los Angeles
4,5,547,3063.280000,2025-06-14 04:28:53,Clothing,PayPal,Los Angeles
...,...,...,...,...,...,...,...
995,996,60,712.470000,2025-01-13 14:16:32,Electronics,Debit Card,New York
996,997,235,2125.180000,2025-06-09 12:37:36,Electronics,PayPal,Chicago
997,998,669,3069.080000,2025-04-01 19:59:01,Home,PayPal,Los Angeles
998,999,931,2262.750000,2025-06-08 01:36:01,Clothing,Debit Card,Online


In [39]:
df.to_sql("transactions",con=engine,index=False, if_exists='replace')


103

In [42]:
df2=pd.read_csv('..\Datasets\order_data 1.csv')
df2.isna().sum()

order_id            0
customer_id         0
order_date          0
order_amount        0
order_status        0
product_category    0
dtype: int64

In [43]:
df2.to_sql("orders",con=engine,index=False,if_exists='replace')

255